In [1]:
import json
from pathlib import Path
import numpy as np

result_file = Path(f"button_usage.jsonl")
if result_file.exists():
    data = list(map(lambda x: json.loads(x), open(result_file).readlines()))

    print(f"The number of collections: {len(data)}")

    learn_costs = np.array([x["learn_costs"] for x in data])
    review_costs = np.array([x["review_costs"] for x in data])

    learn_costs_mean = np.median(learn_costs, axis=0).round(3).tolist()
    review_costs_mean = np.median(review_costs, axis=0).round(3).tolist()
    print(f"Learn costs median: {learn_costs_mean}")
    print(f"Review costs median: {review_costs_mean}")

    first_rating_probs = np.array([x["first_rating_prob"] for x in data])
    review_rating_probs = np.array([x["review_rating_prob"] for x in data])

    first_rating_probs_mean = np.mean(first_rating_probs, axis=0).round(3).tolist()
    review_rating_probs_mean = np.nanmean(review_rating_probs, axis=0).round(3).tolist()
    print(f"First rating prob mean: {first_rating_probs_mean}")
    print(f"Review rating prob mean: {review_rating_probs_mean}")

    first_rating_offsets = np.array([x["first_rating_offset"] for x in data])
    first_session_lens = np.array([x["first_session_len"] for x in data])

    first_rating_offsets_median = (
        np.median(first_rating_offsets, axis=0).round(3).tolist()
    )
    first_session_lens_median = np.median(first_session_lens, axis=0).round(3).tolist()
    print(f"First rating offset median: {first_rating_offsets_median}")
    print(f"First session len median: {first_session_lens_median}")

    forget_rating_offsets = np.array([x["forget_rating_offset"] for x in data])
    forget_session_lens = np.array([x["forget_session_len"] for x in data])

    forget_rating_offsets_median = np.median(forget_rating_offsets).round(3)
    forget_session_lens_median = np.median(forget_session_lens).round(3)
    print(f"Forget rating offset median: {forget_rating_offsets_median}")
    print(f"Forget session len median: {forget_session_lens_median}")

    short_term_recall = np.array([x["short_term_recall"] for x in data])
    short_term_recall = short_term_recall[~(short_term_recall == 0).any(axis=1) & ~(short_term_recall == 1).any(axis=1)]
    short_term_recall_mean = np.mean(short_term_recall, axis=0).round(3).tolist()
    print(f"Short term recall mean: {short_term_recall_mean}")

The number of collections: 9999
Learn costs median: [27.56, 22.53, 13.0, 7.48]
Review costs median: [19.17, 12.7, 7.99, 5.91]
First rating prob mean: [0.21, 0.099, 0.53, 0.161]
Review rating prob mean: [0.227, 0.625, 0.148]
First rating offset median: [-0.63, -0.27, -0.1, 0.0]
First session len median: [1.51, 1.07, 0.7, 0.15]
Forget rating offset median: -0.2
Forget session len median: 0.7
Short term recall mean: [0.739, 0.888, 0.912, 0.844]
